In [1]:
import requests
import sys, os
from dotenv import load_dotenv
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
from security_routes.utilities import hash_password
from pathlib import Path

## Base URL

You can run this notebook locally, on main runner, or development runner, please uncomment the one you are going to use, and comment the other base url's

In [2]:
# Main runner
# base_url = "https://tmv9bz5v4q.us-east-1.awsapprunner.com/"
# Development runner
# base_url = "https://cp3by92k8p.us-east-1.awsapprunner.com/"
# Local
base_url = "http://localhost:8000/"
prefix = "latest"

In [3]:
hash_password("password")

'$2b$12$b9VHZpN6q4GDeQmYOqCKK.eFZXrFoLrZLT6VtrEscYKmw79VsF8S.'

In [5]:
load_dotenv("../.env", override=True)

True

In [6]:
adminpassword = os.getenv("ADMIN_PASSWORD")
adminpassword is not None

True

In [7]:
response = requests.post(
    base_url+"latest/token", data={"username": "admin", "password": os.getenv("ADMIN_PASSWORD")}
)

admin_token = response.json()["access_token"]

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [163]:
len(admin_token)

145

Create user and group

In [164]:
new_user_data = {
        "username": "new_user",
        "email": "new_user@example.com",
        "is_admin": False,
    }

new_auth_data = {
    "username": "new_user",
    "password": "password123",
}

response = requests.post(
        base_url + f"{prefix}/users",
        params=new_user_data,
        data=new_auth_data,
        headers={"Authorization": f"Bearer {admin_token}"},
    )

response

<Response [400]>

In [165]:
response.text

'{"detail":"Username already registered"}'

In [166]:
group_data = {
    "name": "new_group",
    "description": "A new group",
}
response = requests.post(
    base_url+f"{prefix}/groups",
    params=group_data,
    headers={"Authorization": f"Bearer {admin_token}"},
)
response

<Response [400]>

In [167]:
link_data = {
    "username": "new_user",
    "groupname": "new_group",
}
response = requests.post(
    base_url+f"{prefix}/link-user-group",
    params=link_data,
    headers={"Authorization": f"Bearer {admin_token}"},
)
response

<Response [400]>

Create a revision

In [168]:
response = requests.post(
    base_url+"latest/token", data={"username": "new_user", "password": "password123"}
)
token = response.json()["access_token"]

In [169]:
len(token)

151

In [170]:
new_version_data = {
    "name": "New Version",
    "iso_language": "eng",
    "iso_script": "Latn",
    "abbreviation": "NV",
    "rights": "Some Rights",
    "machineTranslation": False,
    "is_reference": True,
}

response = requests.post(
    base_url+f"{prefix}/version",
    json=new_version_data,
    headers={"Authorization": f"Bearer {token}"},
)
version_id =response.json()["id"]
response.json()

{'id': 1208,
 'name': 'New Version',
 'iso_language': 'eng',
 'iso_script': 'Latn',
 'abbreviation': 'NV',
 'rights': 'Some Rights',
 'forward_translation_id': None,
 'back_translation_id': None,
 'machineTranslation': False,
 'owner_id': 382,
 'group_ids': [],
 'is_reference': True}

Update Name

In [171]:
# Modify name of the version
attr_update = {
    "id": version_id,
    "name": "New Version 2"
    }
response = requests.put(
    base_url+f"{prefix}/version",
    json=attr_update,
    headers={"Authorization": f"Bearer {token}"},
)
version_name = response.json()["name"]
version_name

'New Version 2'

In [172]:
print(new_version_data)

{'name': 'New Version', 'iso_language': 'eng', 'iso_script': 'Latn', 'abbreviation': 'NV', 'rights': 'Some Rights', 'machineTranslation': False, 'is_reference': True}


In [173]:
response.text

'{"name":"New Version 2","iso_language":"eng","abbreviation":"NV","forward_translation_id":null,"machine_translation":false,"deleted":false,"deletedAt":null,"id":1208,"iso_script":"Latn","rights":"Some Rights","back_translation_id":null,"is_reference":true,"owner_id":382}'

In [174]:
headers = {"Authorization": f"Bearer {token}"}
test_revision = {
    "version_id": version_id,
    "name": "Test Revision",
}
test_upload_file = Path("../fixtures/uploadtest.txt")

with open(test_upload_file, "rb") as file:
    files = {"file": file}
    response = requests.post(
    base_url + f"{prefix}/revision", params=test_revision, files=files, headers=headers
    )
revision_id =response.json()["id"]
response.json()

{'id': 1869,
 'bible_version_id': 1208,
 'version_abbreviation': 'NV',
 'date': '2024-10-02',
 'name': 'Test Revision',
 'published': False,
 'back_translation_id': None,
 'machineTranslation': False,
 'iso_language': 'eng',
 'is_reference': False}

In [175]:
headers = {"Authorization": f"Bearer {token}"}
test_revision = {
    "version_id": version_id,
    "name": "Test Revision",
}
test_upload_file = Path("../fixtures/uploadtest.txt")

with open(test_upload_file, "rb") as file:
    files = {"file": file}
    response = requests.post(
    base_url + f"{prefix}/revision", params=test_revision, files=files, headers=headers
    )
other_revision_id =response.json()["id"]
response.json()

{'id': 1870,
 'bible_version_id': 1208,
 'version_abbreviation': 'NV',
 'date': '2024-10-02',
 'name': 'Test Revision',
 'published': False,
 'back_translation_id': None,
 'machineTranslation': False,
 'iso_language': 'eng',
 'is_reference': False}

In [176]:
response = requests.get(
    base_url+f"{prefix}/revision",
    headers={"Authorization": f"Bearer {token}"},
)
response.json()

[{'id': 1217,
  'bible_version_id': 842,
  'version_abbreviation': 'SS-DEL',
  'date': '2024-08-05',
  'name': None,
  'published': False,
  'back_translation_id': None,
  'machineTranslation': False,
  'iso_language': 'swh',
  'is_reference': False},
 {'id': 1220,
  'bible_version_id': 842,
  'version_abbreviation': 'SS-DEL',
  'date': '2024-08-05',
  'name': None,
  'published': False,
  'back_translation_id': None,
  'machineTranslation': False,
  'iso_language': 'swh',
  'is_reference': False},
 {'id': 8,
  'bible_version_id': 20,
  'version_abbreviation': 'german_hoffnung_für_alle',
  'date': '2024-03-01',
  'name': 'HFA',
  'published': False,
  'back_translation_id': None,
  'machineTranslation': False,
  'iso_language': 'deu',
  'is_reference': False},
 {'id': 9,
  'bible_version_id': 21,
  'version_abbreviation': 'french_louis_segond',
  'date': '2024-03-01',
  'name': 'LBS21',
  'published': False,
  'back_translation_id': None,
  'machineTranslation': False,
  'iso_language'

In [177]:
response = requests.get(
    base_url+f"{prefix}/version",
    headers={"Authorization": f"Bearer {token}"},
)
response.json()

[{'id': 1168,
  'name': 'New Version 2',
  'iso_language': 'eng',
  'iso_script': 'Latn',
  'abbreviation': 'NV',
  'rights': 'Some Rights',
  'forward_translation_id': None,
  'back_translation_id': None,
  'machineTranslation': False,
  'owner_id': 382,
  'group_ids': [473],
  'is_reference': True},
 {'id': 1169,
  'name': 'New Version',
  'iso_language': 'eng',
  'iso_script': 'Latn',
  'abbreviation': 'NV',
  'rights': 'Some Rights',
  'forward_translation_id': None,
  'back_translation_id': None,
  'machineTranslation': False,
  'owner_id': 382,
  'group_ids': [473],
  'is_reference': True},
 {'id': 1170,
  'name': 'New Version 2',
  'iso_language': 'eng',
  'iso_script': 'Latn',
  'abbreviation': 'NV',
  'rights': 'Some Rights',
  'forward_translation_id': None,
  'back_translation_id': None,
  'machineTranslation': False,
  'owner_id': 382,
  'group_ids': [473],
  'is_reference': True},
 {'id': 1171,
  'name': 'New Version 2',
  'iso_language': 'eng',
  'iso_script': 'Latn',
  '

Add Assessment

In [186]:
assessment_data = {
        "revision_id": revision_id,
        "reference_id": other_revision_id,
        "type": "word-alignment",
    }
response = requests.post(
            base_url+f"{prefix}/assessment",
            params=assessment_data,
            headers={"Authorization": f"Bearer {admin_token}"},
        )
assert response.status_code == 200

In [187]:
response.text

'[{"id":2498,"revision_id":1869,"reference_id":1870,"type":"semantic-similarity","status":"queued","requested_time":"2024-10-02T14:08:16.138458","start_time":null,"end_time":null,"owner_id":115}]'

Delete version and revision

In [23]:
response = requests.delete(
    base_url+f"{prefix}/revision",
    params={"id": revision_id},
    headers={"Authorization": f"Bearer {token}"},
)
response.json()

{'detail': 'Revision 19 deleted successfully.'}

In [24]:
response = requests.delete(
    base_url+f"{prefix}/version",
    params={"id": version_id},
    headers={"Authorization": f"Bearer {token}"},
)
response.json()

{'detail': 'Version New Version 2 successfully deleted.'}

Delete user and group

In [25]:
response = requests.delete(
    base_url+f"{prefix}/users",
    params={"username": "new_user"},
    headers={"Authorization": f"Bearer {admin_token}"},
)
response

<Response [204]>

In [26]:
response = requests.delete(
    base_url+f"{prefix}/groups",
    params={"groupname": "new_group"},
    headers={"Authorization": f"Bearer {admin_token}"},
)
response

<Response [204]>

In [29]:
headers = {"Authorization": f"Bearer {admin_token}"}
response = requests.get(f"{base_url}v3/language", headers=headers)
assert response.status_code == 200
assert response.json() == [{"iso639": "eng", "name": "english"}]

AssertionError: 